## 1. Authenticating user

In [2]:
import requests
import json

token = '<token>'
instance = 'odm-demos.genestack.com'
app_endpoint = 'https://%s/frontend/endpoint/application/invoke/genestack/' % instance
url_authenticate = app_endpoint + 'signin/authenticateByApiToken'

s = requests.Session()
s.post(url_authenticate, json=[token])

<Response [200]>

## 2. Sharing a study

In [2]:
# Get a list of user groups and their accessions
url_get_groups = app_endpoint + 'groupsadmin/getData'
groups = {}
for group in json.loads(s.post(url=url_get_groups).text)['result']['groups']:
    groups[group['name']] = group['accession']
print groups

{u'Genestack members': u'GSG000002', u'Curators': u'GSG000004'}


In [3]:
# Share a study to a specific user group
study = 'GSF202527'
group = groups['Genestack members']
url_share_study = app_endpoint + 'shareutils/shareFiles'
s.post(url=url_share_study, json=[[study], [group], ["VIEW", "EDIT"]])

<Response [200]>

## 3. Exporting a study

In [4]:
# Create an export file
url_export_study = app_endpoint + 'file-export/createFile'
url_initialize = app_endpoint + 'initializer/initialize'
exported_file = json.loads(s.post(url=url_export_study, json=[[study]]).text)['result']
s.post(url=url_initialize, json=[[exported_file]])
print exported_file

GSF202540


In [5]:
# Check export status
url_export_status = app_endpoint + 'initializer/loadInfo'
status = json.loads(s.post(url=url_export_status, json=[[exported_file]]).text)['result'][0]['initializationStatus']['id']
print status

COMPLETE


In [6]:
# When status == "COMPLETE", get the download URL
url_export_link = app_endpoint + 'file-export/getFileInfo'
export_base_url = 'https://%s/frontend/endpoint/application/download/genestack/file-export/' % instance
print export_base_url + json.loads(s.post(url=url_export_link, json=[exported_file]).text)['result']['downloadString']

https://genestack.research.corteva.com/frontend/endpoint/application/download/genestack/file-export/c3RvcmFnZTpleHBvcnRlZC8yMDE5LzEyLzA5LzM4MjVjYmRiLTFhMTMtNGM4OS1iNzA4LTYxNzRiZTRiZmQ5MC9FeHBvcnRlZCUyMG9uJTIwMjAxOS0xMi0wOSUyMDE2XzQ4XzAxLnppcC9hL3QvZy9jL2FwcGxpY2F0aW9uL3ppcC9hL3QvZy9jLzIyMjU=/Exported on 2019-12-09 16_48_01.zip


## 4. Deleting a study, a sample, or an expression object

In [6]:
object_to_delete = 'GSF015445'
url_delete_object = app_endpoint + 'arvados-importer/wipeStudy'
s.post(url=url_delete_object, json=[object_to_delete])

<Response [200]>